In [0]:
GALAXY = 'ngc6822'
GALAXYFULLNAME = 'ngc6822'
USER = 'dobos'
PREFIX = '/datascope/subaru'
USEPMAP = False

#date of observation
YEAR = 2025
MONTH = 7
DAY = 1

OBS_PATH = f'{PREFIX}/data/cmdfit/dSph'
HSC_FILE = f'{PREFIX}/data/cmdfit/dSph/{GALAXY}_tpall3e_g24.cat'
GAIA_FILE = f'{PREFIX}/data/cmdfit/dSph/gaia.h5'
SKY_FILE = f'{PREFIX}/data/cmdfit/dSph/sky_{GALAXYFULLNAME}.feather'
FLUXSTD_FILE = f'{PREFIX}/data/cmdfit/dSph/fluxstd_{GALAXYFULLNAME}.feather'
MLCLASS_FILE = f'{PREFIX}/data/targeting/dSph/{GALAXY}/{GALAXYFULLNAME}_mlclass.csv'
PMAP_FILE = f'{PREFIX}/data/cmdfit/run/{GALAXY}/sim/nobin_chab_nb_250k_001/pmap.h5'
OUTPUT_PATH = f'{PREFIX}/user/{USER}/netflow'
ISOCHRONES_PATH = f'{PREFIX}/data/cmdfit/isochrones/dartmouth/import/afep0_cfht_sdss_hsc'

GAIA_CROSSMATCH_RADIUS = 0.1    # in arcsec

NVISITS = 1
OUTPUT_PATH = f'{PREFIX}/user/{USER}/netflow/{GALAXY}_{NVISITS}_visit'

In [0]:
import os, sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from matplotlib.gridspec import GridSpec
from scipy.special import logsumexp
from scipy.interpolate import interp1d

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord
from astropy.time import Time

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
if 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', 5698))
    debug = True

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

# Compile data

## Plot definitions

In [0]:
import pfs.utils
from pfs.ga.targeting.targets.dsph import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow
from pfs.ga.targeting.io import DataFrameSerializer

In [0]:
galaxy = GALAXIES[GALAXY]
hsc = galaxy.get_photometry()
cmd = galaxy.get_cmd()
ccd = galaxy.get_ccd()
gaia_cmd = galaxy.get_cmd(Gaia)

In [0]:
pointings = galaxy.get_pointings(SubaruPFI)
pointing = pointings[0]

for p in pointings:
    print(p.ra, p.dec, p.posang)

In [0]:
wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

## Load isochrones

In [0]:
from pfs.ga.isochrones.isogrid import IsoGrid

In [0]:
iso = IsoGrid()
iso.load(os.path.join(ISOCHRONES_PATH, 'isochrones.h5'))

In [0]:
iso.values.keys()

## Load probability map

In [0]:
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.selection import ProbabilityCut, ProbabilitySampling

In [0]:
if USEPMAP:
    pmap = ProbabilityMap(cmd.axes)
    pmap.load(PMAP_FILE)

In [0]:
if USEPMAP:
    f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

    l0 = cmd.plot_probability_map(axs[0], pmap, 0)
    axs[0].set_title("non-member")

    l1 = cmd.plot_probability_map(axs[1], pmap, 1)
    axs[0].set_title("member")

    f.tight_layout()

In [0]:
if USEPMAP:
    # Define a cut based on probability

    probcut = ProbabilityCut(pmap, 1, np.log(0.001))

## Load observations

In [0]:
from pfs.ga.targeting.io import TextObservationReader
from pfs.ga.targeting.util.astro import *

In [0]:
obs = SubaruHSC.text_observation_reader().read(HSC_FILE)
obs.data.shape

In [0]:
obs.data.columns

### Plot observations with color cuts

In [0]:
cmap = plt.get_cmap('tab10')

mask = galaxy.get_selection_mask(obs, nb=True)

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, obs, c='r', mask=mask, cmap=cmap)

ax = f.add_subplot(gs[1])
ccd.plot_observation(ax, obs, c='lightgray')
ccd.plot_observation(ax, obs, c='r', mask=mask, cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, obs, c='r', mask=mask, cmap=cmap)

f.suptitle('Stars selected based on color cuts.')

## Get GAIA data

In [0]:
from pfs.ga.targeting.io import Hdf5ObservationReader, Hdf5ObservationWriter, GaiaReader

In [0]:
if os.path.isfile(GAIA_FILE):
    print('GAIA data file found, read from local.')
    r = Hdf5ObservationReader()
    gaia = r.read(GAIA_FILE, GALAXY, 'gaia')
    gaia.frame = 'icrs'
    gaia.equinox = 'J2015'
    print(gaia.data.shape)
else:
    print('GAIA data file not found, running query against archive.')
    r = GaiaReader()
    gaia = r.cone_search(galaxy.pos, galaxy.rad)
    print(gaia.data.shape)

    w = Hdf5ObservationWriter()
    w.write(gaia, GAIA_FILE, GALAXY, 'gaia')

In [0]:
gaia.data.columns

In [0]:
# Cross-match HSC with GAIA
gaia_idx, separation = obs.cross_match(gaia)
print('median separation', np.median(separation.arcsec))
hsc_gaia_mask = (separation.arcsec < GAIA_CROSSMATCH_RADIUS)
print(hsc_gaia_mask.sum())

In [0]:
# Cross-match HSC with GAIA
hsc_idx, separation = gaia.cross_match(obs)
print('median separation', np.median(separation.arcsec))
gaia_hsc_mask = (separation.arcsec < GAIA_CROSSMATCH_RADIUS)
print(gaia_hsc_mask.sum())

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(6, 3), dpi=240)
gs = f.add_gridspec(1, 2, width_ratios=[2, 3], wspace=0.4)   

ax = f.add_subplot(gs[0])
gaia_cmd.plot_observation(ax, gaia, c='lightgray')
gaia_cmd.plot_observation(ax, gaia, c='r', mask=gaia_hsc_mask, cmap=cmap)

ax = f.add_subplot(gs[1], projection=wcs.wcs)
fov.plot_observation(ax, gaia, c='lightgray')
fov.plot_observation(ax, gaia, c='r', mask=gaia_hsc_mask, cmap=cmap)

f.suptitle('GAIA stars with HSC counterparts within 1"')

In [0]:
cmap = plt.get_cmap('viridis')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, obs, c='r', mask=hsc_gaia_mask, cmap=cmap)

ax = f.add_subplot(gs[1])
ccd.plot_observation(ax, obs, c='lightgray')
ccd.plot_observation(ax, obs, c='r', mask=hsc_gaia_mask, cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, obs, c='r', mask=hsc_gaia_mask, cmap=cmap)

f.suptitle('HSC stars with GAIA counterparts within 1"')

## Merge Gaia proper motions into HSC catalog

In [0]:
# Cross-match HSC with GAIA
hsc_gaia_idx, separation = obs.cross_match(gaia)
print('median separation', np.median(separation.arcsec))
hsc_gaia_mask = (separation.arcsec < 1)

hsc_gaia_mask.shape, hsc_gaia_mask.sum()

In [0]:
gaia.data.columns

In [0]:
columns = ['parallax', 'pm', 'pmdec', 'pmra', 'err_parallax', 'err_pmdec', 'err_pmra']
obs.merge(gaia, hsc_gaia_idx, columns=columns, mask=hsc_gaia_mask)

In [0]:
obs.data.columns

## Assign probabilities

In [0]:
# Hard cuts on magnitudes

selection = galaxy.get_selection_mask(obs, nb=True) #, probcut=probcut)
obs.data.shape, selection.size, selection.sum()

In [0]:
# Look up membership probability

#galaxy.assign_probabilities(obs, pmap, mask=selection)
obs.data['p_member'] = 1
(~np.isnan(obs.data['p_member'])).sum()

In [0]:
cmap = plt.get_cmap('viridis')

mask = selection & ~np.isnan(obs.data['p_member'])

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 4], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, obs, c=obs.data['p_member'][mask], mask=mask, cmap=cmap)

ax = f.add_subplot(gs[1])
ccd.plot_observation(ax, obs, c='lightgray')
ccd.plot_observation(ax, obs, c=obs.data['p_member'][mask], mask=mask, cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
l = fov.plot_observation(ax, obs, c=obs.data['p_member'][mask], mask=mask, cmap=cmap)

f.colorbar(l, ax=ax, label='membership probability')

f.suptitle('Stars selected by color cuts, colored by membersphip probability')

In [0]:
hist, bins = np.histogram(obs.data['p_member'][mask])
plt.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')
plt.xlabel('p')
plt.ylabel('frequency')
plt.title('Distribution of membership probability')

## Assign priorities

In [0]:
# Calculate priorities and required exposure time
# This is implemented differently for every galaxy

galaxy.assign_priorities(obs, mask=None) # , mask=selection)
obs.data['priority'].unique()

In [0]:
obs.data['exp_time']

In [0]:
cmap = plt.get_cmap('tab10')

obs_mask = obs.data['priority'] >= 0

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 4], wspace=0.4)   

ax = f.add_subplot(gs[0])
cmd.plot_observation(ax, obs, c='lightgray')
cmd.plot_observation(ax, obs, c=obs.data['priority'][obs_mask], mask=obs_mask, cmap=cmap)

ax = f.add_subplot(gs[1])
ccd.plot_observation(ax, obs, c='lightgray')
ccd.plot_observation(ax, obs, c=obs.data['priority'][obs_mask], mask=obs_mask, cmap=cmap)

ax = f.add_subplot(gs[2], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
l = fov.plot_observation(ax, obs, c=obs.data['priority'][obs_mask], mask=obs_mask, cmap=cmap)

f.colorbar(l, ax=ax, label='target priority')

f.suptitle('Priority class')

In [0]:
obs_mask.size, obs_mask.sum()

In [0]:
# Plot distribution of priorities

f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

hist = np.bincount(obs.data['priority'][obs_mask])
ax.bar(np.arange(hist.size - 1), hist[:-1])

# hist = np.bincount(obs.data['priority'][obs_assigned & (obs.data['priority'] >= 0) & (obs.data['priority'] < 9)])
# plt.bar(np.arange(hist.size), hist, color='r')


ax.set_title('Priority class number distribution')
ax.set_xlabel('Priority class')
ax.set_ylabel('Target count')


In [0]:
# Plot distribution of required visits

f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

hist = np.bincount((np.ceil(obs.data['exp_time'][obs_mask & (obs.data['priority'][obs_mask] < 9)] / 1800.0)).astype(int))
ax.bar(np.arange(hist.size), hist)

# hist = np.bincount(obs.data['priority'][obs_assigned & (obs.data['priority'] >= 0) & (obs.data['priority'] < 9)])
# plt.bar(np.arange(hist.size), hist, color='r')


ax.set_title('Distribution of required visit (t_exp = 1800 s)')
ax.set_xlabel('Number of required visits')
ax.set_ylabel('Target count')


In [0]:
# Save the catalog with p_member and priority

fn = os.path.join(OUTPUT_PATH, f'{GALAXY}_obs.feather')
print(fn)

s = DataFrameSerializer()
s.write(obs.data, fn)

In [0]:
obs.data.columns

## Load sky and flux standards

In [0]:
from pfs.ga.targeting.io import FeatherSkyReader, FeatherFluxStdReader

In [0]:
r = FeatherSkyReader()
sky = r.read(SKY_FILE)
sky.data.shape

In [0]:
# r = FeatherFluxStdReader()
# fluxstd = r.read(FLUXSTD_FILE)
# fluxstd.data.shapefrom pfs.ga.targeting.selection import MagnitudeSelection, ColorSelection
import copy

fstars = MagnitudeSelection(cmd.axes[1], 18, 20).apply(obs) & ColorSelection(cmd.axes[0], 0.09, 0.46).apply(obs) & (obs.data['cli'] <= 0.5) & (obs.data['clg'] <= 0.5)
fluxstd = copy.deepcopy(obs)
fluxstd._set_data(obs.get_data(mask=fstars))
fluxstd.data

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='b')
fov.plot_observation(ax, sky, c='lightgray')

ax.set_title(f'Sky positions (count={len(sky)})')

# f.tight_layout()

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0], projection=wcs.wcs)
fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, fluxstd, c='r')

ax.set_title(f'Flux standards, count={len(fluxstd)}')

# f.tight_layout()

# Define the netflow problem

In [0]:
# r = Angle(50, u.arcmin)

# Filter flux standards around pointing
# obs_mask = galaxy.get_selection_mask(obs, nb=True)
# obs_mask &= obs.cone_search(p.pos, r)
# obs_mask[obs_mask] &= (obs.data['priority'][obs_mask] >= 0) & \
#                       (obs.data['priority'][obs_mask] <= 9) & \
#                       ~np.isnan(obs.data['exp_time'][obs_mask])

print('obs_mask', obs_mask.size, obs_mask.sum())

In [0]:
# Reduce number of sky fibers
sky_mask = np.full(sky.shape, True)
# sky_mask = sky.cone_search(p.pos, r)
# sky_mask &= sky.random_sample(0.3)

print('sky_mask', sky_mask.size, sky_mask.sum())

In [0]:
# Filter flux standards around pointing
fluxstd_mask = np.full(fluxstd.shape, True)
# fluxstd_mask = fluxstd.cone_search(p.pos, r)

print('fluxstd_mask', fluxstd_mask.size, fluxstd_mask.sum())

In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0], projection=wcs.wcs)
# fov.plot_observation(ax, obs, c='lightgray')
fov.plot_observation(ax, obs, c='b', mask=obs_mask)
fov.plot_observation(ax, sky, c='lightgray', mask=sky_mask)
fov.plot_observation(ax, fluxstd, c='r', mask=fluxstd_mask)

pfi = SubaruPFI(instrument_options={ 'layout': 'calibration' })
for p in galaxy.get_pointings(SubaruPFI)[:]:
    pfi.plot_focal_plane(ax, fov, corners=True, projection=SubaruWFC(p))

# f.tight_layout()

In [0]:
ncobras = SubaruPFI().bench.cobras.nCobras
ncobras

In [0]:
obs.data.columns

In [0]:
# Create and instrument

instrument_options = dict(
    # Instrument layout mode
    layout = 'calibration',     # Use calibration product
    # layout = 'full',          # Use default settings

    # Temp directory for cobra coach output
    cobra_coach_dir = os.path.expandvars('/tmp/$USER/cobra_coach'),

    # Cobra coach module version
    # cobra_coach_module_version = None,

    # FPI configuration
    # instdata_path = None,

    # Black dot list
    # blackdots_path = None,

    # Black dot radius margin
    # black_dot_radius_margin = 1.0,

    # List of spectrograph modules to use
    # spectrograph_modules = [1, 2, 3, 4],
)

instrument = SubaruPFI(instrument_options=instrument_options)

In [0]:
# Construct the netflow problem

gurobi_options = dict(
    seed=0,                 # random seed
    presolve=2,             # agressiveness of presolve which tries to eliminate variables from the LP problem
    method=3,               # 3 means concurrent, 4 means deterministic concurrent
    degenmoves=0,           # degenerate simplex moves, set to 0 to prevent too much time to be spent on trying to improve the current solution
    heuristics=0.5,         # how much of the time to spend by performing heuristics
    mipfocus=1,             # mipfocus=1 is balanced toward finding more feasible solutions
                            # mipfocus=2 is balanced toward proving that the current solution is the best
                            # mipfocus=3 is to be used when the objection bound is moving very slowly
    mipgap=0.01,            # relative stopping criterion for bounds on the objective
    LogToConsole=1,         # 
    timelimit=300           # in sec
)

target_classes = {
    'sky': dict(
        prefix = 'sky',
        min_targets = 240,
        max_targets = 320,
        non_observation_cost = 0,
    ),
    'cal': dict(
        prefix = 'cal',
        min_targets = 40,
        max_targets = 240,
        non_observation_cost = 0,
        calib = True,
    ),
}
    
for i in range(10):
    target_classes[f'sci_P{i}'] = dict(
        prefix = 'sci',
        min_targets = None,
        max_targets = None,
        non_observation_cost = max(20 - 2 * i, 1),
        partial_observation_cost = 1e5,
    )

target_classes[f'sci_P0']['non_observation_cost'] = 1000
target_classes[f'sci_P1']['non_observation_cost'] = 500
target_classes[f'sci_P2']['non_observation_cost'] = 200
target_classes[f'sci_P3']['non_observation_cost'] = 100
target_classes[f'sci_P4']['non_observation_cost'] = 100
target_classes[f'sci_P5']['non_observation_cost'] = 100
target_classes[f'sci_P6']['non_observation_cost'] = 100
target_classes[f'sci_P7']['non_observation_cost'] = 50
target_classes[f'sci_P8']['non_observation_cost'] = 10
target_classes[f'sci_P9']['non_observation_cost'] = 0
    
########
    
cobra_groups = {
        'location': dict(
            groups = np.random.randint(4, size=ncobras),
            target_classes = [ 'sky' ],
            min_targets = 40,
            max_targets = 80,
            non_observation_cost = 10,
        ),
        'instrument': dict(
            groups = np.random.randint(4, size=ncobras),
            target_classes = [ 'sky' ],
            min_targets = 10,
            max_targets = 25,
            non_observation_cost = 10,
        )
    }

#######

netflow_options = dict(
    # Penalize cobra moves with respect to cobra center
    cobra_move_cost = lambda dist: 0.1 * dist,

    # Add a penalty if the target is too close to a black dot
    # black_dot_penalty = lambda dist: max(0, dot_penalty * (1 - 0.5 * dist)),

    fiber_non_allocation_cost = 1e5,

    collision_distance = 2.0,
    elbow_collisions = True,
    # forbidden_targets = [
    #     43218108431
    # ],
    # forbidden_pairs = [
    #     [43486543901, 43218108431],
    # ],

    target_classes = target_classes,
    
    # cobra_groups = cobra_groups,

    # time_budgets = {
    #     'science': dict(
    #         target_classes = [ 'sci_P0', 'sci_P1', 'sci_p2' ],
    #         budget = 5  # hr
    #     )
    # },

    num_reserved_fibers = 0,

    # This will only be used when netflow is rewritten to step-by-step fiber assignment
    # constrain_already_observed = False,

    # Allow more visits than minimally required
    allow_more_visits = True,

    epoch = 2016, # all catalogs must match
    ignore_proper_motion = True,

    # Generate full gurobi variable names instead of numbered ones (slow to build problem)
    use_named_variables = True,
)

debug_options = dict(
    ignoreEndpointCollisions = False,
    ignoreElbowCollisions = False,
    ignoreForbiddenPairs = False,
    ignoreForbiddenSingles = False,
    ignoreCalibTargetClassMinimum = False,
    ignoreCalibTargetClassMaximum = False,
    ignoreScienceTargetClassMinimum = False,
    ignoreScienceTargetClassMaximum = False,
    ignoreTimeBudget = False,
    ignoreCobraGroupMinimum = False,
    ignoreCobraGroupMaximum = False,
    ignoreReservedFibers = False,
)

pointings = []
exp_time = 12 / NVISITS * 900

# Observation time is the next day from now at midnight
obs_time = datetime(YEAR, MONTH, DAY, hour=0, minute=30, second=0, microsecond=0)+pd.Timedelta(hours=10)
obs_time = Time(obs_time)
print('obs_time', obs_time)

for p in galaxy.get_pointings(SubaruPFI)[:]:
    # Need to convert targeting.pointing to netflow.pointing
    pointings.append(Pointing(
        p.ra, p.dec,
        posang=p.posang,
        # obs_time=Time(datetime.now()),
        obs_time=obs_time,
        exp_time = exp_time,
        nvisits=NVISITS
    ))

nf = Netflow(f'{GALAXY}', instrument, pointings, workdir=OUTPUT_PATH,
             netflow_options=netflow_options,
             solver_options=gurobi_options,
             debug_options=debug_options)

# nf.append_science_targets(obs, mask=(obs.data['priority'] <= 9), exp_time=visits * p.exp_time)    ## All targets with priority
# nf.append_science_targets(obs, mask=(obs.data['priority'] <= 9))    ## All targets with priority
# nf.append_science_targets(obs, mask=obs_mask)    ## Only selected targets with priority
# nf.append_science_targets(obs, mask=obs_mask, exp_time=visits * p.exp_time)
# nf.append_science_targets(obs, mask=(obs.data['priority'] <= 9), exp_time=visits * p.exp_time)

nf.append_science_targets(obs, mask=obs_mask)
nf.append_sky_targets(sky, mask=sky_mask)
nf.append_fluxstd_targets(fluxstd, mask=fluxstd_mask)

len(nf.targets)

In [0]:
for c in nf.targets.columns:
    print(c, nf.targets[c].dtype)

In [0]:
nf.targets

In [0]:
nf.targets[nf.targets['prefix'] == 'sky']

In [0]:
# Plot the final target list loaded into netflow
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0], projection=wcs.wcs)

mask = nf.targets['prefix'] == 'sky'
fov.plot(ax, nf.targets['RA'][mask], nf.targets['Dec'][mask], native_frame='world', c='b')
mask = nf.targets['prefix'] == 'cal'
fov.plot(ax, nf.targets['RA'][mask], nf.targets['Dec'][mask], native_frame='world', c='r')
mask = nf.targets['prefix'] == 'sci'
fov.plot(ax, nf.targets['RA'][mask], nf.targets['Dec'][mask], native_frame='world', c='y')


pfi = SubaruPFI(instrument_options={ 'layout': 'calibration' })
for p in galaxy.get_pointings(SubaruPFI)[:]:
    pfi.plot_focal_plane(ax, fov, corners=True, projection=SubaruWFC(p))

# f.tight_layout()

In [0]:
import cProfile
profiler = cProfile.Profile()
profiler.enable()

In [0]:
nf.build()

In [0]:
nf.targets.shape, nf._Netflow__target_cache_mask.shape, nf._Netflow__target_cache_mask.sum()

In [0]:
for fp_pos in nf._Netflow__target_fp_pos:
    print(fp_pos.shape)

In [0]:
### This accesses internal variables, do not use this method to read the solution!

# TODO: this might be broken with multiple pointings

# Plot the focal plane positions


cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0])

for i, fp_pos in enumerate(nf._Netflow__target_fp_pos):
    pmask = nf._Netflow__target_mask[i]
    
    mask = nf.targets['prefix'][pmask] == 'sky'
    fp.plot(ax, fp_pos[mask].real, fp_pos[mask].imag, native_frame='pixel', c='b')
    mask = nf.targets['prefix'][pmask] == 'cal'
    fp.plot(ax, fp_pos[mask].real, fp_pos[mask].imag, native_frame='pixel', c='r')
    mask = nf.targets['prefix'][pmask] == 'sci'
    fp.plot(ax, fp_pos[mask].real, fp_pos[mask].imag, native_frame='pixel', c='y')


pfi = SubaruPFI(instrument_options={ 'layout': 'calibration' })
for p in galaxy.get_pointings(SubaruPFI)[:]:
    pfi.plot_focal_plane(ax, fp, corners=True, projection=SubaruWFC(p))

# f.tight_layout()

In [0]:
profiler.disable()

In [0]:
profiler.print_stats(sort='tottime')

In [0]:
nf.save_problem()

In [0]:
nf._Netflow__solver_options['timelimit'] = 1800

In [0]:
nf.solve()

In [0]:
nf.save_solution()

In [0]:
nf.visits[0].visit_idx

In [0]:
### This accesses internal variables, do not use this method to read the solution!

# Plot the final target list loaded into netflow
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0], projection=wcs.wcs)

ra = nf._Netflow__target_cache.ra
dec = nf._Netflow__target_cache.dec
fov.plot(ax, ra, dec, native_frame='world', c='gray')

for visit in nf.visits:
    tidx = np.array(list(nf._Netflow__target_assignments[visit.visit_idx].keys()))
    ra = nf._Netflow__target_cache.ra[tidx]
    dec = nf._Netflow__target_cache.dec[tidx]
    fov.plot(ax, ra, dec, native_frame='world', c='r', ms=1)

pfi = SubaruPFI(instrument_options={ 'layout': 'calibration' })
for p in galaxy.get_pointings(SubaruPFI)[:]:
    pfi.plot_focal_plane(ax, fov, corners=True, projection=SubaruWFC(p))

# f.tight_layout()

In [0]:
### This accesses internal variables, do not use this method to read the solution!

# Plot the final target list loaded into netflow
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0])

ra = nf._Netflow__target_cache.ra
dec = nf._Netflow__target_cache.dec
# fp.plot(ax, ra, dec, native_frame='world', c='gray')

for visit in nf.visits:
    tidx = np.array(list(nf._Netflow__target_assignments[visit.visit_idx].keys()))
    # Map target indices to target cache indices
    cache_map = np.where(nf._Netflow__target_mask[visit.visit_idx])[0]
    fp_map = nf._Netflow__cache_to_fp_pos_map[visit.visit_idx]

    # ra = nf._Netflow__target_cache.ra[tidx]
    # dec = nf._Netflow__target_cache.dec[tidx]

    p = nf._Netflow__target_fp_pos[visit.visit_idx][fp_map[tidx]]
    fp.plot(ax, p.real, p.imag, native_frame='pixel', c='r', ms=0.5)

    break

pfi = SubaruPFI(instrument_options={ 'layout': 'calibration' })
for p in galaxy.get_pointings(SubaruPFI)[:]:
    pfi.plot_focal_plane(ax, fp, corners=True, projection=SubaruWFC(p))
    pfi.plot_cobras(ax, fp)

# f.tight_layout()

### Extract results

In [0]:
from pfs.ga.targeting.io import DataFrameSerializer

In [0]:
assignments = nf.get_target_assignments(include_target_columns=True,
                                        include_unassigned_fibers=True,
                                        include_engineering_fibers=True)

fn = os.path.join(OUTPUT_PATH, f'{GALAXY}_assignments.feather')
print(fn)

s = DataFrameSerializer()
s.write(assignments, fn)

for c in assignments.columns:
    print(c, assignments[c].dtype)

In [0]:
assignments

In [0]:
mask = assignments['prefix'] == 'sky'
np.array(assignments['RA'][mask])

In [0]:
# Plot the final target list loaded into netflow
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0], projection=wcs.wcs)

mask = assignments['prefix'] == 'sky'
fov.plot(ax, assignments['RA'][mask], assignments['Dec'][mask], native_frame='world', c='b', ms=1)
mask = assignments['prefix'] == 'cal'
fov.plot(ax, assignments['RA'][mask], assignments['Dec'][mask], native_frame='world', c='r', ms=1)
mask = assignments['prefix'] == 'sci'
fov.plot(ax, assignments['RA'][mask], assignments['Dec'][mask], native_frame='world', c='y', ms=1)


pfi = SubaruPFI(instrument_options={ 'layout': 'calibration' })
for p in galaxy.get_pointings(SubaruPFI)[:]:
    pfi.plot_focal_plane(ax, fov, corners=True, projection=SubaruWFC(p))

# f.tight_layout()

In [0]:
summary = nf.get_target_assignment_summary()

fn = os.path.join(OUTPUT_PATH, f'{GALAXY}_summary.feather')
print(fn)

s = DataFrameSerializer()
s.write(summary.reset_index(), fn)

summary

In [0]:
# All targets observed at least once
summary[summary['num_visits'] > 0]

# Look at the ILP constraints to find any issues

In [0]:
model = nf._Netflow__problem._GurobiProblem__model
constrs = model.getConstrs()
len(constrs)

In [0]:
# Total number of constraints
len(nf._Netflow__constraints.all)

In [0]:
q = 0
for i in range(len(constrs)):
    name = constrs[i].ConstrName
    sense = constrs[i].Sense
    slack = constrs[i].Slack
    rhs = constrs[i].RHS

    #if slack == 0.0 and rhs > 1:
    if sense != '=' and rhs > 1:
        print(name, sense, rhs, slack)

        q += 1
        if q == 300:
            break

In [0]:
# Edges adding significant terms to the cost
# There shouldn't be any target sinks here, cause that would mean
# partially observed targets which we want to avoid

cost = nf._Netflow__problem._GurobiProblem__cost
n = cost.size()

print('total cost', cost.getValue())

for i in range(cost.size()):
    var = cost.getVar(i)
    name = var.VarName
    value = var.X
    coeff = cost.getCoeff(i)
    if value != 0 and coeff > 0:
        print(name, value, coeff, value * coeff)